In [2]:
# After installing this restart your colab
!pip install tpu_tf2

In [4]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tpu_tf2.tpu import strategy
seed = tf.random.set_seed(2021)

In [5]:
print("Number of replica for TPU:", strategy.num_replicas_in_sync)

Number of replica for TPU: 8


In [6]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
seed
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

170508288/170498071 [==============================] - 2s 0us/step


In [7]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 10))

In [8]:
seed
with strategy.scope():
  model = tf.keras.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=5)])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [9]:
seed
model.fit(x_train, y_train, epochs = 10, batch_size = 512, validation_data=(x_test, y_test), use_multiprocessing=True, workers=10)

Epoch 1/10
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


98/98 [==============================] - 11s 67ms/step - loss: 4.2052 - accuracy: 0.1744 - top_k_categorical_accuracy: 0.6441 - val_loss: 1.9267 - val_accuracy: 0.2940 - val_top_k_categorical_accuracy: 0.8156
Epoch 2/10
98/98 [==============================] - 3s 29ms/step - loss: 1.7523 - accuracy: 0.3581 - top_k_categorical_accuracy: 0.8632 - val_loss: 1.6995 - val_accuracy: 0.3804 - val_top_k_categorical_accuracy: 0.8810
Epoch 3/10
98/98 [==============================] - 3s 28ms/step - loss: 1.5397 - accuracy: 0.4404 - top_k_categorical_accuracy: 0.9072 - val_loss: 1.5839 - val_accuracy: 0.4204 - val_top_k_categorical_accuracy: 0.9062
Epoch 4/10
98/98 [==============================] - 3s 28ms/step - loss: 1.4356 - accuracy: 0.4838 - top_k_categorical_accuracy: 0.9239 - val_loss: 1.4097 - val_accuracy: 0.4833 - val_top_k_categorical_accuracy: 0.9306
Epoch 5/10
98/98 [==============================] - 3s 29ms/step - loss: 1.3612 - accuracy: 0.5121 - top_k_categorical_accuracy: 0.932

In [10]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 8s 21ms/step - loss: 1.1467 - accuracy: 0.5919 - top_k_categorical_accuracy: 0.9566


[1.1467190980911255, 0.5918999910354614, 0.9565999507904053]